In [23]:
# Update following paths (including the one in load_data) to your local paths
# Synch your data with the remote server
# Update the csv file whit all information on runs

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize as op
from scipy.stats import linregress
from scipy.optimize import curve_fit
import plotly.express as px
import statsmodels.api as sm
import numpy as np
from scipy.optimize import curve_fit
import os
from IPython.display import display, HTML
import re
from docx import Document
from docx.shared import Inches
from docx.shared import RGBColor
from io import BytesIO
dir_path = r"C:\Users\louis\OneDrive\Bureau\Stage Tokyo\data_Spring8\runs900410to902083"
folder = r"C:\Users\louis\OneDrive\Bureau\Stage Tokyo\data_Spring8"
folder_path_runs = 'C:/Users/louis/OneDrive/Bureau/Stage Tokyo/data_Spring8/docs_runs/'

In [24]:
def load_current_Scan(runstart, runend):
    df0 = pd.read_csv(dir_path+'/run'+str(runstart)+'.csv')
    SXMOKE_On = []
    SXMOKE_Off = []
    SXFUN_On = []
    SXFUN_Off = []
    for ii in range(runstart, runend + 1):
        df0 = pd.read_csv(f'/Users/louis/OneDrive/Bureau/Stage Tokyo/data_Spring8/runs900410to902083/run{ii}.csv')
    # 清理列名前后的空格
        df0.columns = df0.columns.str.strip()
        df0['On'] = df0['On'].fillna(0)
        for kk in range(len(df0)):
            I_raw = df0['I_raw'].iloc[kk]
            I_norm = df0['I_norm'].iloc[kk]
            I0_GM2 = df0['I0(GM2)'].iloc[kk]
            On_val = df0['On'].iloc[kk]
            if -1 < I_raw < 0.4 and I0_GM2 > -1:
                if On_val > 0.1:
                    df_MOKE_Off = I_raw
                    df_fun_Off = I0_GM2
                    SXMOKE_Off.append(df_MOKE_Off)
                    SXFUN_Off.append(df_fun_Off)
                elif On_val <= 0.1:
                    df_MOKE_On = I_raw
                    df_fun_On = I0_GM2
                    SXMOKE_On.append(df_MOKE_On)
                    SXFUN_On.append(df_fun_On)
    return np.array(SXMOKE_On), np.array(SXMOKE_Off), np.array(SXFUN_On),np.array(SXFUN_Off)   


In [ ]:
# Creating the docx file
# Select the runs you want to study and fill in the parameters
run_id_first = 905718 # first run id
run_id_last = 905750 # last run id
# ensure validity
if run_id_first > run_id_last:
    raise ValueError("run_id_first should be lower than run_id_last")
output_path = folder_path_runs+str(run_id_first)+'-'+str(run_id_last)+'_runs.docx'
if os.path.exists(output_path):
    raise FileExistsError(f"{output_path} already exists. Execution stopped.")
print("Creating the docx file for runs {} to {}".format(run_id_first, run_id_last))
## Open google sheet
df = pd.read_csv(folder+'\\202505_orbital_MOKE - 工作表1.csv')
mask = df[df.columns[0]].astype(str).str.startswith('Current measurement#')
df_current_measurements = df[mask]
start_run_arr = df_current_measurements['start run#'].values.tolist()
end_run_arr = df_current_measurements['end run#'].values.tolist()
# Don't forget to change these parameters
Resistance_value = 195
AT1 = 'Zr 0.5 um'
AT2 = 'Zr 0.15 um'
AT1b = ''
AT2b = 'Zr 1.0 um'
AT3b = ''
AT4b = ''
doc = Document()
doc.add_heading('Report on MOKE measurements', level=1)
doc.save(output_path)
run_id_list = np.arange(run_id_first, run_id_last+1).tolist()
run_id_list_real = [x for x in run_id_list if x in start_run_arr and x in end_run_arr]
for run_id in run_id_list_real:
    doc = Document(output_path)
    scan_value = df_current_measurements.loc[df_current_measurements['start run#'] == run_id, 'Scan'].values
    photon_energy_value = df_current_measurements.loc[df_current_measurements['start run#'] == run_id, 'Photon Energy'].values
    run_value = run_id
    current_value = df_current_measurements.loc[df_current_measurements['start run#'] == run_id, 'Current (mA)'].values
    Gain_value = df_current_measurements.loc[df_current_measurements['start run#'] == run_id, 'Gain settting'].values
    MCP_value = df_current_measurements.loc[df_current_measurements['start run#'] == run_id, 'MCP Voltage'].values
    print('Adding run : {} of photon energy {} eV'.format(run_id, photon_energy_value[0]))
    SXMOKE_On, SXMOKE_Off, SXFUN_On, SXFUN_Off = load_current_Scan(run_id, run_id)
    On_x = np.linspace(0, 500, len(SXMOKE_On))
    Off_x = np.linspace(0, 500, len(SXMOKE_Off))
    # Remove NaN/inf
    mask_on = np.isfinite(SXFUN_On) & np.isfinite(SXMOKE_On)
    mask_off = np.isfinite(SXFUN_Off) & np.isfinite(SXMOKE_Off)
    x_on = SXFUN_On[mask_on]
    y_on = SXMOKE_On[mask_on]
    x_off = SXFUN_Off[mask_off]
    y_off = SXMOKE_Off[mask_off]
    # Fit On
    slope_on, intercept_on, *_ = linregress(x_on, y_on)
    y_fit_on = slope_on * x_on + intercept_on
    jitter_on = np.std(y_on - y_fit_on)
    # Fit Off
    slope_off, intercept_off, *_ = linregress(x_off, y_off)
    y_fit_off = slope_off * x_off + intercept_off
    jitter_off = np.std(y_off - y_fit_off)
    paragraph = doc.add_paragraph()
    run = paragraph.add_run("Run#  "+str(run_value))
    run.add_break() 
    run.add_text(str(scan_value[0]))
    run.add_break() 
    run.add_text("Event Per Step: 10000")
    run.add_break()
    run.add_text("Scan Mode: sync")
    run.add_break()
    run.add_text("Retry Mode: restart")
    run.add_break()
    run.add_text("Scan Config File:##")
    run.add_break()
    run.add_text("Pulse Selector Setting: XFEL(Continuous; 60 Hz)")
    run.add_break()
    run2 = paragraph.add_run("Current Intensity : "+str(current_value[0]))
    run2.add_break()
    run2.add_text("Resisstance: {} Om".format(Resistance_value))
    run2.add_break()
    run2.font.color.rgb = RGBColor(255, 0, 0)
    run3 = paragraph.add_run("in EH4a")
    run3.add_break()
    run3.add_text("photo energy : "+str(photon_energy_value[0]))
    run3.add_break()
    run3.add_text("AT1: "+AT1)
    run3.add_break()
    run3.add_text("AT2: "+AT2)
    run3.add_break()
    run3.add_text("in OH")
    run3.add_break()
    run3.add_text("AT1: "+AT1b)
    run3.add_break()
    run3.add_text("AT2: "+AT2b)
    run3.add_break()
    run3.add_text("AT3: "+AT3b)
    run3.add_break()
    run3.add_text("AT4: "+AT4b)
    run3.add_break()
    run3.add_text("Gain : "+str(Gain_value[0]))
    run3.add_break()
    run3.add_text("MCP : "+str(MCP_value[0]))
    run3.add_break()
    run3.add_text(f"On:  slope={slope_on:.5f}, intercept={intercept_on:.5f}, jitter={jitter_on:.5f}")
    run3.add_break()
    run3.add_text(f"Off: slope={slope_off:.5f}, intercept={intercept_off:.5f}, jitter={jitter_off:.5f}")
    # first dot graph
    buf = BytesIO()
    plt.scatter(On_x, SXMOKE_On, color='black', label='MOKE On')
    plt.scatter(Off_x, SXMOKE_Off, color='red', label='MOKE Off')
    plt.title('Scan Checking')
    plt.xlabel('scan')
    plt.ylabel('Intensity')
    plt.savefig(buf, format='png')
    plt.close()
    doc.add_picture(buf, width=Inches(4))
    # second dot graph
    buf2 = BytesIO()
    plt.figure(figsize=(8,5))
    plt.scatter(x_on, y_on, color='black', s=10, label='On')
    plt.scatter(x_off, y_off, color='red', s=10, label='Off')
    plt.plot(x_on, y_fit_on, '--', color='black', label='Fit On')
    plt.plot(x_off, y_fit_off, '--', color='red', label='Fit Off')
    plt.title('Moke Scan: On vs Off')
    plt.xlabel('Fundamental Intensity')
    plt.ylabel('MOKE Intensity')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(buf2, format='png')
    plt.close()
    doc.add_picture(buf2, width=Inches(4))
    doc.save(output_path)
print("Document saved")


Creating the docx file for runs 905685 to 905717
Adding run :  905685
Adding run :  905686
Adding run :  905687
Adding run :  905688
Adding run :  905689
Adding run :  905690
Adding run :  905691
Adding run :  905692
Adding run :  905693
Adding run :  905694
Adding run :  905695
Adding run :  905696
Adding run :  905697
Adding run :  905698
Adding run :  905699
Adding run :  905700
Adding run :  905701
Adding run :  905702
Adding run :  905703
Adding run :  905704
Adding run :  905705
Adding run :  905706
Adding run :  905707
Adding run :  905708
Adding run :  905709
Adding run :  905710
Adding run :  905711
Adding run :  905712
Adding run :  905713
Adding run :  905714
Adding run :  905715
Adding run :  905716
Adding run :  905717
Document saved
